In [1]:
import pandas as pd

In [56]:
tables = pd.read_html("http://www.basketball-reference.com/leagues/NBA_2016_games.html")

In [65]:
games = tables[0]
games.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,,.1,Notes
0,"Tue, Oct 27, 2015",8:00 pm,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,NaN
1,"Tue, Oct 27, 2015",8:00 pm,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,NaN
2,"Tue, Oct 27, 2015",10:30 pm,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,NaN
3,"Wed, Oct 28, 2015",7:30 pm,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,NaN
4,"Wed, Oct 28, 2015",7:30 pm,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,NaN


In [53]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 9 columns):
Date               38 non-null object
Start (ET)         38 non-null object
Visitor/Neutral    38 non-null object
PTS                38 non-null int64
Home/Neutral       38 non-null object
PTS.1              38 non-null int64
                   38 non-null object
 .1                2 non-null object
Notes              0 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 2.8+ KB


In [66]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}
games = (games.dropna(thresh = 4)
              .rename(columns = column_names)[['date','away_team','away_points','home_team','home_points']]
              .assign(date=lambda x: pd.to_datetime(x['date']))
              .set_index('date', append=True)
              .rename_axis(["game_id", "date"])
              .sort_index())

In [67]:
games.head()

,,away_team,away_points,home_team,home_points
game_id,date,,,,
0,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94
1,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111
3,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112
4,2015-10-28,Chicago Bulls,115,Brooklyn Nets,100


In [68]:
games = (pd.melt(games.reset_index(), id_vars = ['game_id', 'date'], value_vars = ['away_team', 'home_team'], value_name ='team'))

In [69]:
games.head()

,game_id,date,variable,team
0,0,2015-10-27,away_team,Detroit Pistons
1,1,2015-10-27,away_team,Cleveland Cavaliers
2,2,2015-10-27,away_team,New Orleans Pelicans
3,3,2015-10-28,away_team,Philadelphia 76ers
4,4,2015-10-28,away_team,Chicago Bulls


In [70]:
games.groupby('team')['date'].diff().dt.days -1

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     0.0
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    0.0
15    NaN
16    NaN
17    NaN
18    0.0
19    NaN
20    1.0
21    NaN
22    NaN
23    1.0
24    1.0
25    NaN
26    1.0
27    NaN
28    1.0
29    NaN
     ... 
46   -2.0
47   -3.0
48    NaN
49   -3.0
50    NaN
51   -4.0
52   -3.0
53   -4.0
54   -3.0
55    0.0
56    0.0
57   -3.0
58    2.0
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
64    1.0
65    1.0
66    1.0
67    1.0
68    1.0
69    1.0
70    1.0
71    1.0
72    2.0
73    2.0
74    2.0
75    0.0
Name: date, dtype: float64

In [73]:
games['rest'] = games.sort_values('date').groupby('team')['date'].diff().dt.days -1
games.dropna().head()

,game_id,date,variable,team,rest
4,4,2015-10-28,away_team,Chicago Bulls,0.0
8,8,2015-10-28,away_team,Cleveland Cavaliers,0.0
14,14,2015-10-28,away_team,New Orleans Pelicans,0.0
17,17,2015-10-29,away_team,Memphis Grizzlies,0.0
18,18,2015-10-29,away_team,Dallas Mavericks,0.0
